# Phonon Workflow Tutorial with Force Fields

We start with imports necessary to test the tutorial automatically. In practice, you can load a structure file from any other place and you are also not required to generate the data in a temporary directory.

In [ ]:
import tempfile
import warnings
from pathlib import Path

tmp_dir = tempfile.mkdtemp()
TEST_ROOT = Path().cwd().parent.parent / "tests"
TEST_DIR = TEST_ROOT / "test_data"

First, we load a structure from a file.

In [ ]:
from pymatgen.core.structure import Structure

si_structure = Structure.from_file(TEST_DIR / "structures" / "Si.cif")

Then, we load the `PhononMaker` and run_locally to perform the calculation directly here in the notebook.

In [ ]:
from jobflow import run_locally

from atomate2.forcefields.flows.phonons import PhononMaker

warnings.filterwarnings("ignore")

flow = PhononMaker(
    min_length=3.0,
    born_maker=None,
    use_symmetrized_structure="conventional",
    create_thermal_displacements=False,
    store_force_constants=False,
    prefer_90_degrees=False,
    generate_frequencies_eigenvectors_kwargs={"tstep": 100},
).make(si_structure)
run_locally(flow, create_folders=True, raise_immediately=True, root_dir=tmp_dir)

One can switch to a different force field as well!

In [ ]:
from atomate2.forcefields.jobs import ForceFieldRelaxMaker, ForceFieldStaticMaker

warnings.filterwarnings("ignore")

flow = PhononMaker(
    min_length=3.0,
    use_symmetrized_structure="conventional",
    create_thermal_displacements=False,
    store_force_constants=False,
    prefer_90_degrees=False,
    generate_frequencies_eigenvectors_kwargs={"tstep": 100},
    static_energy_maker=ForceFieldStaticMaker(force_field_name="MACE_MP_0B3"),
    bulk_relax_maker=ForceFieldRelaxMaker(force_field_name="MACE_MP_0B3"),
    phonon_displacement_maker=ForceFieldStaticMaker(force_field_name="MACE_MP_0B3"),
).make(si_structure)

run_locally(flow, create_folders=True, raise_immediately=True, root_dir=tmp_dir)

Or by using the name only:

In [ ]:
PhononMaker.from_force_field_name(force_field_name="MACE_MP_0B3")
run_locally(flow, create_folders=True, raise_immediately=True, root_dir=tmp_dir)

Now, we clean up the temporary directory that we made. In reality, you might want to keep this data.

In [ ]:
import shutil

shutil.rmtree(tmp_dir)